# Maximum Entropy Model

## Principle of Maximum Entropy
Entropy: $$H(P) = - \sum_x{P(x)\log{P(x)}}$$
$$0 \leq H(P) \leq \log{|X|}$$
where $|X|$ is the number of possible values in $X$. If and only if the probability distribution of $X$ is uniform distribution, the information entropy would be equal to its maximum possible value, $\log {|X|}$

## Definition
Given a training data: 

$$T = {(x_1, y_1), (x_2, y_2), ... , (x_N, y_N)}$$


Empirical Probability Distribution:
$$\tilde{P}(x, y) = \frac{v(x, y)}{N}  $$
where $N$ is the size of training dataset, $v(x, y)$ is the number of times that $(x, y)$ occurs in training dataset.


We introduce the following feature function:
$$
  f(x, y) = 
  \begin{cases}
                                   1 & \text{if $(x, y)$ occurs in training dataset} \\
                                   0 & \text{otherwise} \\
  \end{cases}
$$

The expected value of $f(x, y)$ with respect to empirical probability distribution $\tilde{P}(x, y)$ is equal to:
$$\mathbb{E}_{\tilde{P}}(f) = \sum_{x, y}{\tilde{P}(x, y) f(x, y)}$$

The expected value of $f(x, y)$ with respect to the model $P(y|x)$ is equal to:
$$\mathbb{E}_P(f) = \sum_{x, y}{ \tilde{P}(x) P(y|x) f(x, y) }$$
where $\tilde{P}(x)$ is the empirical distribution of $x$ in the training dataset and it is usually set equal to $\frac{1}{N}$

By constraining the expected value to be the equal to the empirical value, we have that:
$$\mathbb{E}_P(f) = \mathbb{E}_{\tilde{P}}(f)$$
$$\sum_{x, y}{ \tilde{P}(x) P(y|x) f(x, y) } = \sum_{x, y}{\tilde{P}(x, y) f(x, y)} $$

We have so many constrains as the number of feature functions.

According to the principle of Maximum Entropy, we should select the model that is as close as possible to uniform. In other words, we should select the model $P^*$ with Maximum Entropy:
$$P^* = \arg\max_{P \in C} ( - \sum_{x, y} \tilde{P}(x) P(y|x) \log P(y|x) )$$
Given that:
1. $P(y|x) \geq 0$ for all $(x, y)$
2. $\sum_{y}{P(y|x)} = 1$ for all $x$
3. $\sum_{x, y}{ \tilde{P}(x) P(y|x) f(x, y) } = \sum_{x, y}{\tilde{P}(x, y) f(x, y)} $ for all feature functions

To solve the above optimization problem, we introduce Lagrangian multipliers. We focus on the unconstrained dual problem and we estimate the lambda free variables $\{\lambda_1, \lambda_2, ... , \lambda_n\}$ with the Maximum Likelihood Estimation method.

It can be proven that if we find the $\{\lambda_1, \lambda_2, ... , \lambda_n\}$ parameters which maximize the dual problem, the probability given a $x$ to be classified as $y$ is equal to:

$$P(y|x) = \frac{\exp (\sum_{i=1}^n \lambda_i f_i(x, y))}{  \sum_y{    \exp (\sum_{i=1}^n \lambda_i f_i(x, y)) }   }$$


### Estimate the lambda parameters
We use **IIS (Improved Iterative Scaling)** to estimate lambda parameters.

Algorithm:
![](http://blog.datumbox.com/wp-content/uploads/2013/11/maxentropy_tutorial/image014.png)

The $f^\#(x, y)$ is the total number of feature which are active for a particular $(x, y)$ pair. If the number is constant, then the $\Delta{\lambda_i}$ can be calculated in close-form:
$$\Delta{\lambda_i} = \frac{1}{M} \log {\frac{\mathbb{E}_{\tilde{P}}(f_i)}{    \mathbb{E}_P(f_i)   }}$$
where $M$ is equal to $f^\#(x, y)$
## References:
+ [Machine Learning Tutorial: The Max Entropy Text Classifier](https://blog.datumbox.com/machine-learning-tutorial-the-max-entropy-text-classifier/)
+ [codes](https://github.com/WenDesi/lihang_book_algorithm/blob/master/maxENT/maxENT.py)

## Codes

+ $P(y|x)$ is computed for 1 sample
+ In most cases, we do not explicitly multiply $f(x, y)$ because of its definition
+ $\mathbb{E}_P(f)$ maps from $(x, y)$ pair to its probability. It is computed by traverse all the elements in the training data matrix

### 1. Import packages

In [0]:
import math
import numpy as np
import pandas as pd
import cv2
import tqdm
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

### 2. Prepare data

In [0]:
def binarization(img):
    bin_img = img.astype(np.uint8)
    cv2.threshold(bin_img, 50, 1, cv2.THRESH_BINARY_INV, bin_img) # pixel = 0 if value > 50 else 1
    return bin_img

raw_data = pd.read_csv('/content/gdrive/My Drive/data/train_binary.csv', header=0) # binary classification
data = raw_data.values
imgs = data[0:, 1:] # for one row, the first column is the label followed by the image data
labels = data[:, 0]

# reduce the size of training dataset to train faster
# imgs = imgs[:1000] # (1000, 784)
# labels = labels[:1000] # (1000,)

# binarization
for index, img in enumerate(imgs):
    imgs[index] = binarization(img)
    
# preprocess: add subscript information to each pixel
new_imgs = list()
for img in imgs:
    pixels = list()
    for index, pixel in enumerate(img):
        pixels.append(str(index) + '_' + str(pixel))
    new_imgs.append(pixels)
imgs = new_imgs # new_imgs: (1000, 784)
# print(imgs[0]) # each element in img is a string
    
# choose 33% of samples for training, and the rest for testing
x_train, x_test, y_train, y_test = train_test_split(imgs, labels, test_size=0.33, random_state=23323)

### 3. Build model

In [0]:
class MaximumEntropy(object):
    def fit(self, features, labels):
        # store the size of training dataset
        self.N = len(features)
        
        # compute the set of possible labels
        self.labels = set(labels) 
        
        # compute the empirical probability distribution: $\tilde{P}(x, y)$
        tilde_p_xy = defaultdict(int)
        for feature, label in zip(features, labels):
            for x in feature:
                tilde_p_xy[(x, label)] += 1 / self.N
            
        # store the number of (x, y) pair in training dataset
        self.n = len(tilde_p_xy)
        
        # M is equal to then number of (x, y)
        self.M = self.n
            
        # define feature function: $f(x, y)$
        self.f_xy = lambda x, y: 1 if (x, y) in tilde_p_xy else 0
        
        # give id to each (x, y) pair that occurs in training dataset
        self.id2xy = list(tilde_p_xy.keys())
        self.xy2id = {xy: id for id, xy in enumerate(self.id2xy)}
        
        # compute the expected value of f(x, y) with respect to empirical probability distribution: $\mathbb{E}_{\tilde{P}}(f)$
        e_tilde_p_f = tilde_p_xy # since the feature function is defined as 1 if (x, y) pair appears
        
        # initialize lambdas with zeros
        self.lambdas = [0.0] * self.n # each lambda corresponds to 1 (x, y) pair
        # start iteration
        for epoch in tqdm.tqdm(range(100)):
            # compute the expected value of f(x, y) with respect to P(y|x)
            e_p_f = defaultdict(float)
            
            for feature in features:
                p_yx = self.get_p_yx(feature)
                for x in feature: # traverse all the elements of training data matrix
                    for y in self.labels:
                        e_p_f[(x, y)] += (1 / self.N) * p_yx[y] if self.f_xy(x, y) else 0 # the feature belongs to only 1 class, so (x, y) might not exist
                    
            deltas = list()
            for i in range(self.n): # focus on single lambda
                xy = self.id2xy[i]
                # calculate the increment of lambdas
                delta = 1 / self.M * math.log(e_tilde_p_f[xy] / e_p_f[xy])
                deltas.append(delta)
            # update lambdas
            self.lambdas = [self.lambdas[i] + deltas[i] for i in range(self.n)]

    def get_p_yx(self, feature):
        '''
            return P(y|x) 
            
            Note: P(y|x) is computed based on 1 sample
        '''
        y2probs = defaultdict(float)
        for x in feature: # simply add all the lambdas whose (x, y) pair appears in this sample
            for y in self.labels:
                if (x, y) in self.xy2id.keys():
                    y2probs[y] += self.lambdas[self.xy2id[(x, y)]]
        for y in self.labels:
            y2probs[y] = math.exp(y2probs[y])
            
        Z = sum(y2probs.values())
        return {y: y2probs[y] / Z for y in self.labels}
        
    def predict(self, feature):
        y2probs = self.get_p_yx(feature)
        return max([(prob, y) for y, prob in y2probs.items()])[1]


### 4. Train and Test



In [4]:
me = MaximumEntropy()
me.fit(x_train, y_train)
def predict(features): # root.predict receive 1D array, thus we need to traverse batch
    y_predicted = list()
    for feature in features:
        y_pred = me.predict(feature)
        y_predicted.append(y_pred)
    return np.asarray(y_predicted)

y_predicted = predict(x_test)
score = accuracy_score(y_predicted, y_test)
print(score)

100%|██████████| 100/100 [1:54:47<00:00, 67.86s/it]


0.9660894660894661
